In [3]:
import pandas as pd
import numpy as np
import csv
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import TaggedDocument

In [4]:
wnl = WordNetLemmatizer()

class Sentences(object):
    
    def __init__(self, filename, column):
        self.filename = filename
        self.column = column
        
    @staticmethod
    def get_tokens(text):
        """Helper function for tokenizing data"""
        return [wnl.lemmatize(r.lower()) for r in text.split()]
 
    def __iter__(self):
        reader = csv.DictReader(open(self.filename, 'r' ))
        for row in reader:
            words = self.get_tokens(row[self.column])
            tags = ['%s|%s' % (row['Artist'], row['SongID'])]
            yield TaggedDocument(words=words, tags=tags)

In [5]:
filename = 'C:/Users/alienware/Desktop/660project/preprocess/lyrics_dfclean_rap100.csv'
sentences = Sentences(filename=filename, column='Lyrics')

# for song lookups
df_train = pd.read_csv(filename)

In [6]:

from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec(
    alpha=0.002,
    min_alpha=0.002,
    workers=15, 
    min_count=2,
    window=10,
    size=300,
    iter=20,
    sample=0.001,
    negative=5
)

C:\LearnPython\lib\site-packages\gensim\models\doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
C:\LearnPython\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [7]:
model.build_vocab(sentences)

In [8]:
model.save('lyrics.doc2vec')

model = Doc2Vec.load('lyrics.doc2vec')

In [9]:
model.wv.word_vec('rap')

array([ 9.22648003e-04,  1.02066959e-03,  2.26430551e-04, -8.50012526e-04,
       -5.64071874e-04, -5.03094052e-04, -1.20561686e-03,  1.43142091e-03,
        1.48837653e-03, -2.08818921e-04,  1.33928040e-03, -1.42352446e-03,
       -1.09329587e-04,  9.19446931e-04,  1.46354013e-03,  5.10462734e-04,
       -1.47717854e-03,  7.11222412e-04, -1.63055817e-03,  1.50625754e-04,
        7.16709706e-04,  7.60422554e-04, -7.43660028e-04,  1.30806339e-03,
       -1.61521486e-03, -1.08269560e-04, -1.16585742e-03, -1.03447167e-03,
       -1.28136040e-03, -8.67220631e-04, -1.60373119e-03, -1.62033574e-03,
        2.79905740e-04, -1.23875693e-03, -1.61456075e-04, -1.02278376e-04,
       -7.65314791e-04, -5.02639276e-04, -7.57477712e-04,  2.24551317e-04,
        1.42786698e-03, -5.77597122e-04, -6.55181182e-04,  1.01301703e-03,
        7.38075352e-04, -3.81955499e-04,  1.59585569e-03, -1.02736265e-03,
        5.71544166e-04, -1.60775660e-03,  9.53817740e-04, -4.43160854e-04,
        8.55688064e-04,  

In [17]:
model.wv.most_similar('brother')

[('sound', 0.18308037519454956),
 ('friendsnthey', 0.18186749517917633),
 ('save', 0.18142272531986237),
 ('else', 0.17627644538879395),
 ('xanax', 0.17517533898353577),
 ('yountell', 0.16326946020126343),
 ('holy', 0.16178609430789948),
 ('jelly', 0.16161216795444489),
 ('near', 0.157787024974823),
 ('talk', 0.15720215439796448)]

In [11]:
def print_titles(results):
    lookup = lambda x: df_train[
        df_train.song_id==int(x)
    ].song.values[0]
    return [
        [
            i[0].split('|')[0], 
            lookup(i[0].split('|')[1]), 
            i[1]
        ] for i in results
    ]

In [12]:
print_titles(
    model.docvecs.most_similar([model['rap']], topn=10)
)

AttributeError: 'DataFrame' object has no attribute 'song_id'